In [ ]:
https://www.kaggle.com/code/rtatman/data-cleaning-challenge-handling-missing-values/notebook

In [2]:
import pandas as pd
import numpy as np

In [4]:
# read in all our data
sf_permits = pd.read_csv("./Building_Permits.csv",low_memory=False)

In [5]:
#Let's check if any data is missing

In [6]:
sf_permits.head()

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
0,201505065519,4,sign - erect,05/06/2015,0326,023,140,NaN,Ellis,St,...,3.0,constr type 3,NaN,NaN,NaN,3.0,Tenderloin,94102.0,"(37.785719256680785, -122.40852313194863)",1380611233945
1,201604195146,4,sign - erect,04/19/2016,0306,007,440,NaN,Geary,St,...,3.0,constr type 3,NaN,NaN,NaN,3.0,Tenderloin,94102.0,"(37.78733980600732, -122.41063199757738)",1420164406718
2,201605278609,3,additions alterations or repairs,05/27/2016,0595,203,1647,NaN,Pacific,Av,...,1.0,constr type 1,1.0,constr type 1,NaN,3.0,Russian Hill,94109.0,"(37.7946573324287, -122.42232562979227)",1424856504716
3,201611072166,8,otc alterations permit,11/07/2016,0156,011,1230,NaN,Pacific,Av,...,5.0,wood frame (5),5.0,wood frame (5),NaN,3.0,Nob Hill,94109.0,"(37.79595867909168, -122.41557405519474)",1443574295566
4,201611283529,6,demolitions,11/28/2016,0342,001,950,NaN,Market,St,...,3.0,constr type 3,NaN,NaN,NaN,6.0,Tenderloin,94102.0,"(37.78315261897309, -122.40950883997789)",144548169992


In [7]:
#Metadata Review

#Let's look at number of missing and populated values for each column and display top 25 columns with missing values. 

missing_values_count = sf_permits.isnull().sum()
non_missing_values_count = sf_permits.notnull().sum()

merged_na_nonna = pd.concat([missing_values_count, non_missing_values_count,sf_permits.dtypes], axis=1).reset_index()
merged_na_nonna.columns=['Field','Count of Missing Values','Count of Filled Values','Data Type']
merged_na_nonna.sort_values(by='Count of Missing Values',ascending=False)[:25]

,Field,Count of Missing Values,Count of Filled Values,Data Type
32,TIDF Compliance,198898,2,object
22,Voluntary Soft-Story Retrofit,198865,35,object
11,Unit Suffix,196939,1961,object
7,Street Number Suffix,196684,2216,object
37,Site Permit,193541,5359,object
19,Structural Notification,191978,6922,object
23,Fire Only Permit,180073,18827,object
10,Unit,169421,29479,float64
17,Completed Date,101709,97191,object
24,Permit Expiration Date,51880,147020,object


In [8]:
#Now let's check percentage of missing values
total_cells = np.product(sf_permits.shape)
total_missing = missing_values_count.sum()

# percent of data that is missing
(total_missing/total_cells) * 100

26.26002315058403

In [9]:
#It appears that over a quarter of cells are empty. Let's explore some columns with many missing values to decide if we should
# drop na value or impute.
#suppress scientific notation for display purposes
np.set_printoptions(suppress=True)
# sf_permits[ "TIDF Compliance"].unique() #unique values: nan, 'Y', 'P'
# sf_permits[ "Voluntary Soft-Story Retrofit"].unique() #unique values: nan, 'Y'
# sf_permits[ "Unit Suffix"].unique() #unique values: 'HOA', 'L', 'B20H', 'COMMERCIAL', "RES'L", 'UPPER', 'G', 'H', "RESID'L", 'FRONT BLDG', 'RETAIL 1', 'RESIDL 3/F', 'P', 'OFFICE 2/F', 'BLDG E',
# sf_permits[ "Street Number Suffix"].unique() #unique values: nan, 'A', 'C', 'V', 'B', 'F', 'R', 'D', '0', 'E', 'P', 'G', 'J', 'K', 'H', 'L', 'I', '½', 'N'
# sf_permits[ "Fire Only Permit"].unique() #unique values:[nan, 'Y']
# sf_permits[ "Unit"].unique() #unique values: 0.,  301.,    1.,  201., 1007.,  401.,  510.,  726.,
# sf_permits[ "Existing Construction Type"].unique() #unique values: 3.,  1.,  5., nan,  2.,  4.
# sf_permits[ "Proposed Construction Type"].unique() #unique values: nan,  1.,  5.,  3.,  4.,  2.
# sf_permits[ "Plansets"].unique() #unique values:  2.,    0.,   nan,    6.,    4.,    3.,   20., 9000.,    1.
sf_permits[ "Site Permit"].unique() #unique values: nan, 'Y'

array([nan, 'Y'], dtype=object)

In [10]:
# Imputation

#For Boolean fields - Site Permit, Fire Only Permit - let's replace nan values with N (false).
sf_permits [["Site Permit","Fire Only Permit","Voluntary Soft-Story Retrofit"]]=sf_permits [["Site Permit","Fire Only Permit","Voluntary Soft-Story Retrofit"]].fillna('N')
sf_permits[ "Fire Only Permit"].unique()


array(['N', 'Y'], dtype=object)

In [11]:
#For these string fields - replace with empty string
cols = ["Voluntary Soft-Story Retrofit","Unit Suffix","Street Number Suffix","Structural Notification","Fire Only Permit", "Unit","Existing Units"]
sf_permits [cols] = sf_permits [cols].fillna('')
sf_permits[ "Unit Suffix"].unique()

array(['', 'A', 'D', 'BLDG 1', 'C', 'B', 'E', 'FRONT', 'F', 'COMML',
       'HOA', 'L', 'B20H', 'COMMERCIAL', "RES'L", 'UPPER', 'G', 'H',
       "RESID'L", 'FRONT BLDG', 'RETAIL 1', 'RESIDL 3/F', 'P',
       'OFFICE 2/F', 'BLDG E', 'BLDG D', 'BLDG C', 'BLDG B', 'BLDG A',
       'PIER 3', 'PIER 2', 'PIER 1', 'J', 'PH-4', 'RESIDENCE', 'C132',
       'REAR UNIT', 'FRONT UNIT', 'REAR', 'LOWER', 'PH2B', 'RESIDENTIA',
       'W', 'REAR BLDG', 'PH', '#R-2', '1A-4', 'THEATRE', 'BLDG. 3',
       'RESDL/REAR', 'T', 'N', 'UTILITY', 'BLDG 3', 'O', 'LOWER UNIT',
       'FRNT RIGHT', 'MIDDLE', 'STORE', 'R', '1/F', '2/F', 'REAR G/F',
       'BLDG 449', "COM'L 1/F", 'CU-3', 'PH-1', 'FRNT BLDG', 'K',
       'LEAS OFFIC', 'COMML-3', 'S', 'M', 'PH1C', "COM'L", 'LEVEL 4',
       'BLD 7', '1503B', 'BLDG 3F', 'TENTATIVE', 'C1', 'CHILD DEV',
       'BASEMENT', '3/F', 'CU-2', 'C102', 'EAST', 'WEST', 'UPPER UNIT',
       'Garage', '0', 'COMML-2', '-A', '-B', 'FRNT ENTRA', 'C-4', 'A102',
       'BLDG 3E', '1ST 

In [15]:
#Review numeric fields to determine which missing values can be pre-filled
#sf_permits[ "Revised Cost"].head(50).unique() #unique values:
sf_permits[ "Estimated Cost"].head(50).unique() #unique values:


array([  4000.,      1.,  20000.,   2000., 100000.,  12000.,     nan,
        30000.,  75000.,  64650.,   7000.,  73000.,  18000.,  14000.])

In [16]:
#Fill missing values in cost field with 0
cols = ["Revised Cost", "Estimated Cost"]
sf_permits [cols] = sf_permits [cols].fillna(0)
sf_permits[ "Estimated Cost"].head(50).unique()

array([  4000.,      1.,  20000.,   2000., 100000.,  12000.,      0.,
        30000.,  75000.,  64650.,   7000.,  73000.,  18000.,  14000.])

In [25]:
#Parsing Dates

#First, let's check the format of date values in the dataframe
date_cols = [col for col in sf_permits.columns if 'Date' in col]	
#print(date_cols)
sf_permits[date_cols]

,Permit Creation Date,Current Status Date,Filed Date,Issued Date,Completed Date,First Construction Document Date,Permit Expiration Date
0,05/06/2015,12/21/2017,05/06/2015,11/09/2015,NaN,11/09/2015,11/03/2016
1,04/19/2016,08/03/2017,04/19/2016,08/03/2017,NaN,08/03/2017,12/03/2017
2,05/27/2016,09/26/2017,05/27/2016,NaN,NaN,NaN,NaN
3,11/07/2016,07/24/2017,11/07/2016,07/18/2017,07/24/2017,07/18/2017,07/13/2018
4,11/28/2016,12/01/2017,11/28/2016,12/01/2017,NaN,11/20/2017,12/01/2018
...,...,...,...,...,...,...,...
198895,12/05/2017,12/05/2017,12/05/2017,12/05/2017,NaN,12/05/2017,NaN
198896,12/05/2017,12/06/2017,12/05/2017,12/06/2017,NaN,12/06/2017,04/06/2018
198897,12/06/2017,12/06/2017,12/06/2017,12/06/2017,NaN,12/06/2017,NaN
198898,12/06/2017,12/06/2017,12/06/2017,12/06/2017,NaN,12/06/2017,NaN


In [26]:
sf_permits[date_cols] = sf_permits[date_cols].fillna('')

In [40]:
sf_permits[str(date_cols)+ ' Parsed'] = pd.to_datetime(sf_permits[date_cols])

sf_permits['Permit Creation Date Parsed'] = pd.to_datetime(sf_permits['Permit Creation Date'])
sf_permits['Current Status Date Parsed'] = pd.to_datetime(sf_permits['Current Status Date'])
sf_permits['Completed Date Parsed'] = pd.to_datetime(sf_permits['Completed Date'])

ValueError: to assemble mappings requires at least that [year, month, day] be specified: [day,month,year] is missing

In [33]:
day_of_month_landslides = sf_permits['Permit Creation Date Parsed'].dt.day
day_of_month_landslides

0          6
1         19
2         27
3          7
4         28
          ..
198895     5
198896     5
198897     6
198898     6
198899     7
Name: Permit Creation Date Parsed, Length: 198900, dtype: int32